# Import Packages

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
%matplotlib inline

# Load Data

In [3]:
df = pd.read_csv("df_train.csv")
#SPY = pd.read_csv("Merged_Cleansed\SPY_train.csv")

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
df.head(3)

,PERMNO,Date,TICKER,Price,Return,Shares Outstanding,Turnover,Dollar Turnover,Current Ratio,Cash Ratio,Quick Ratio,Revenue Growth,COGS Growth,Operating Income Growth,Net Income Growth,EPS Basic Growth,Net Operating Cash Flow Growth,SPY Return,Rf,Var,Cov,Beta,Alpha
0,75844,1999-01-01,POT,63.8750,-0.049130,54241,0.563817,36.013813,1.739693,0.152740,0.920842,-0.210577,-0.204199,-0.256408,-0.386155,-0.384146,0.0,0.044942,0.003792,NaN,NaN,NaN,NaN
1,75844,1999-02-01,POT,60.5000,-0.063017,54253,0.616427,37.293818,1.739693,0.152740,0.920842,-0.210577,-0.204199,-0.256408,-0.386155,-0.384146,0.0,0.036561,0.004338,NaN,NaN,NaN,NaN
2,75844,1999-03-01,POT,56.6875,-0.056229,54253,0.782519,44.359042,1.662684,0.071349,0.854213,0.039843,0.062942,-0.007179,-0.009707,-0.009901,0.0,-0.028939,0.004244,NaN,NaN,NaN,NaN


In [5]:
df = df[df['Price'] >= 5]

# Rank for factor

## Date Processing

In [6]:
# Transform Date into datetime type
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

D:\Anaconda\Setup\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
# Construct Date List
Date = []
for i in range(1995, 2016):
    for j in range(1, 13):
        date = str(i) + '-' + str(j)
        Date.append(date)
        
Date = Date[1:]

## Define Rank Function and Use it to rank for each factor

In [8]:
def percentile_to_score(value):
    if value >= 0.9: value = 1
    elif value >= 0.8: value = 2
    elif value >= 0.7: value = 3
    elif value >= 0.6: value = 4
    elif value >= 0.5: value = 5
    elif value >= 0.4: value = 6
    elif value >= 0.3: value = 7
    elif value >= 0.2: value = 8
    elif value >= 0.1: value = 9
    elif value >= 0.0: value = 10
    return value 

def Rank(df_, factor, factor_rank, Date):
    df = df_.copy()
        
    # First, we select the first date's dataframe and rank
    dataframe = df[Date[0]].copy()
#     dataframe = dataframe.sort_values(by = [factor], axis = 0, ascending = False)  # Sort by Turnover
#     dataframe[factor_rank] = range(len(dataframe))  
#     dataframe[factor_rank] += 1 
    dataframe[factor_rank] = dataframe[factor].rank(pct=True).apply(lambda x: percentile_to_score(x))
    
    for i in range(1, len(Date)):
        DF = df[Date[i]].copy()  # Divide the big df into small DF by date
#         DF = DF.sort_values(by = [factor], axis = 0, ascending = False)  # Sort by Turnover
#         DF[factor_rank] = range(len(DF))  
#         DF[factor_rank] += 1 
        DF[factor_rank] = DF[factor].rank(pct=True).apply(lambda x: percentile_to_score(x))
        dataframe = pd.concat([dataframe, DF])
    
    return dataframe

In [9]:
Factor_List = list(df.columns[5:-6])
Factor_Rank = []

for factor in Factor_List:
    factor_rank = factor + " Rank"
    Factor_Rank.append(factor_rank)
    df = Rank(df, factor, factor_rank, Date)   # Rank for each factor

In [10]:
df.head(3)

,PERMNO,TICKER,Price,Return,Shares Outstanding,Turnover,Dollar Turnover,Current Ratio,Cash Ratio,Quick Ratio,Revenue Growth,COGS Growth,Operating Income Growth,Net Income Growth,EPS Basic Growth,Net Operating Cash Flow Growth,SPY Return,Rf,Var,Cov,Beta,Alpha,Turnover Rank,Dollar Turnover Rank,Current Ratio Rank,Cash Ratio Rank,Quick Ratio Rank,Revenue Growth Rank,COGS Growth Rank,Operating Income Growth Rank,Net Income Growth Rank,EPS Basic Growth Rank,Net Operating Cash Flow Growth Rank
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1995-02-01,75844,POT,35.125,0.021352,42781,1.311143,46.053891,2.681226,0.269828,1.690753,-0.330503,-0.328218,-0.334770,-0.362407,-0.373984,0.622333,0.024591,0.005875,0.001403,0.000872,0.621788,0.003840,1,1,2,4,3,10,10,10,9,9,4
1995-02-01,75615,DFI,8.500,-0.014706,27213,0.405101,3.443354,2.698897,0.662280,1.281431,0.055966,0.061417,0.026729,-9.521589,-9.466667,0.303279,0.024591,0.005875,NaN,NaN,NaN,NaN,6,9,2,2,4,4,4,6,10,10,7
1995-02-01,40694,TXI,31.250,0.000000,12406,0.595276,18.602390,2.799778,0.058225,1.420845,0.009644,-0.011341,0.072856,0.115549,0.129412,1.230227,0.024591,0.005875,0.000723,-0.000774,-1.070816,0.014166,4,3,2,8,3,7,8,5,4,4,2


In [11]:
#df_ = df.copy()
for factor in Factor_List:
    #del df_[factor]
    del df[factor]
    
df.head(3)

,PERMNO,TICKER,Price,Return,Shares Outstanding,SPY Return,Rf,Var,Cov,Beta,Alpha,Turnover Rank,Dollar Turnover Rank,Current Ratio Rank,Cash Ratio Rank,Quick Ratio Rank,Revenue Growth Rank,COGS Growth Rank,Operating Income Growth Rank,Net Income Growth Rank,EPS Basic Growth Rank,Net Operating Cash Flow Growth Rank
Date,,,,,,,,,,,,,,,,,,,,,,
1995-02-01,75844,POT,35.125,0.021352,42781,0.024591,0.005875,0.001403,0.000872,0.621788,0.003840,1,1,2,4,3,10,10,10,9,9,4
1995-02-01,75615,DFI,8.500,-0.014706,27213,0.024591,0.005875,NaN,NaN,NaN,NaN,6,9,2,2,4,4,4,6,10,10,7
1995-02-01,40694,TXI,31.250,0.000000,12406,0.024591,0.005875,0.000723,-0.000774,-1.070816,0.014166,4,3,2,8,3,7,8,5,4,4,2


In [13]:
df.reset_index(inplace=True)
df.to_csv("df_final.csv", index=False)